In [25]:
import pandas as pd

In [26]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
train.shape

(891, 12)

In [28]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [29]:
train.groupby('Survived').size()

Survived
0    549
1    342
dtype: int64

In [30]:
train.value_counts('Survived', normalize=True)

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

In [31]:
#성별 생존율,사망율
total= train.groupby('Sex').size().to_frame('전체인원')
total


,전체인원
Sex,
female,314
male,577


In [32]:
filt = train['Survived']== 1
survived = train[filt]
survived = survived.groupby('Sex').size().to_frame('생존인원')
survived

,생존인원
Sex,
female,233
male,109


In [33]:
gender = total.merge(survived, on='Sex')
gender['생존율'] = gender['생존인원']/gender['전체인원'] *100
gender

,전체인원,생존인원,생존율
Sex,,,
female,314,233,74.203822
male,577,109,18.890815


In [34]:
#티켓 클래스별 생존율
total = train.groupby ('Pclass').size().to_frame('전체인원')
total

,전체인원
Pclass,
1,216
2,184
3,491


In [35]:
filt = train['Survived'] ==1
survived = train[filt]
pclass = survived.groupby('Pclass').size().to_frame('생존인원')


In [36]:
pclass = total.merge(pclass, on='Pclass')
pclass['생존율'] = pclass['생존인원']/pclass['전체인원']
pclass

,전체인원,생존인원,생존율
Pclass,,,
1,216,136,0.629630
2,184,87,0.472826
3,491,119,0.242363


In [37]:
train['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [38]:
min = int(train['Age'].min())
max = int(train['Age'].max())
min, max

(0, 80)

In [39]:
ages = []
step = 20
for age in range (0, int(train['Age'].max()+1), step):
    filt = (train['Age'] >= age) & (train['Age'] < age+step)
    total = train[filt]
    count = train[filt]['Age'].count()

    filt = (train['Age'] >= age) & (train['Age'] < age+step) & (train['Survived']==1)
    count_survived = train[filt]['Age'].count()
    ages.append({
                '나이':f'{age}이상 ~ {age+10}미만',
                '전체인원':count,
                '생존인원':count_survived})
    
df = pd.DataFrame(ages)
df.set_index('나이', inplace=True)
df['생존율'] =df['생존인원']/df['전체인원']
df

,전체인원,생존인원,생존율
나이,,,
0이상 ~ 10미만,164,79,0.481707
20이상 ~ 30미만,387,150,0.387597
40이상 ~ 50미만,137,54,0.394161
60이상 ~ 70미만,25,6,0.240000
80이상 ~ 90미만,1,1,1.000000


In [40]:
df['전체인원'].sum()

np.int64(714)

In [41]:
#탑승항구(Embarked)별 전체인원, 생존인원, 생존율
embarks = []

for port in train['Embarked'].dropna().unique():
    # 전체 인원
    filt_total = (train['Embarked'] == port)
    count = train[filt_total]['Embarked'].count()
    
    # 생존 인원
    filt_survived = filt_total & (train['Survived'] == 1)
    count_survived = train[filt_survived]['Embarked'].count()
    
    embarks.append({
        '탑승항구': port,
        '전체인원': count,
        '생존인원': count_survived
    })

df_emb = pd.DataFrame(embarks)
df_emb.set_index('탑승항구', inplace=True)

# 생존율 계산
df_emb['생존율'] = df_emb['생존인원'] / df_emb['전체인원']
df_emb

,전체인원,생존인원,생존율
탑승항구,,,
S,644,217,0.336957
C,168,93,0.553571
Q,77,30,0.389610


In [43]:
import pandas as pd

In [52]:
group = train.groupby(['Embarked','Pclass']).size()

df= pd.DataFrame()
for idx in group.index:
    df.loc[idx[1], idx[0]] = int(group.loc[idx])

df.index.name = '등급'
df


,C,Q,S
등급,,,
1,85.0,2.0,127.0
2,17.0,3.0,164.0
3,66.0,72.0,353.0
